# Lec 10. CNN with MNIST

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init

import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np
import random
import os

## Data Load

In [2]:
mnist_train = dsets.MNIST(root='data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [3]:
batch_size = 100

train_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                           batch_size = batch_size,
                                           shuffle = False)

test_loader = torch.utils.data.DataLoader(dataset = mnist_test,
                                          batch_size = batch_size,
                                          shuffle = False)

mnist_train.train_data.size(), mnist_test.test_data.size()

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

## Define Model

Convolution Output - N : input, K : kernel, P : paddding (default = 0), S : stride (default = 1)

Out = (N + 2P - K) / S + 1

In [4]:
def c_conv(N, K, P=0, S=1):
    return int((N + 2*P - K) / S + 1)

def c_pool(N, K):
    return int(N/K)

In [5]:
c0 = 28
c1 = c_conv(c0, 5)
c2 = c_conv(c1, 5)
c3 = c_pool(c2, 2)
c4 = c_conv(c3, 5)
c5 = c_pool(c4, 2)

print(c1, c2, c3, c4, c5)

24 20 10 6 3


In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),    # 28 -> 24
            nn.ReLU(),
            
            nn.Conv2d(16,32,5),   # 20
            nn.ReLU(),
            nn.MaxPool2d(2,2),    # 10
            
            nn.Conv2d(32,64,5),   # 6
            nn.ReLU(),
            nn.MaxPool2d(2,2)     # 3
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100), # last MaxPooling layer data size 3*3
            nn.ReLU(),
            nn.Linear(100,10)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)

        return out
    
model = CNN().cuda()     # GPU 사용

In [7]:
# multi-nomial classification - CrossEntropyLoss

loss = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
num_epochs = 10

In [9]:
# Model save & load
model_filename = 'cnn_mnist.pkl'

# if os.path.isfile(model_filename):
#     model.load_state_dict(torch.load(model_filename))
#     print("Model Loaded!")
# else:

for epoch in range(num_epochs):
    total_batch = len(mnist_train) // batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):

        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pred = model(X)
        cost = loss(pred, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % total_batch == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'%(epoch+1, num_epochs, i+1, total_batch, cost.item()))

#     if not os.path.isfile(model_filename):
#         print("Model Saved!")
#         torch.save(model.state_dict(), model_filename)

Epoch [1/10], lter [600/600] Loss: 0.2236
Epoch [2/10], lter [600/600] Loss: 0.1960
Epoch [3/10], lter [600/600] Loss: 0.1879
Epoch [4/10], lter [600/600] Loss: 0.1743
Epoch [5/10], lter [600/600] Loss: 0.1671
Epoch [6/10], lter [600/600] Loss: 0.1747
Epoch [7/10], lter [600/600] Loss: 0.1783
Epoch [8/10], lter [600/600] Loss: 0.1528
Epoch [9/10], lter [600/600] Loss: 0.1515
Epoch [10/10], lter [600/600] Loss: 0.1393


## Test

In [10]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

correct = correct.cpu().numpy()
print('correct :', correct)
print('total   :', total)
print('Accuracy of test images: %f %%' % (100 * correct / total))

correct : 9898
total   : 10000
Accuracy of test images: 98.980000 %
